In [1]:
import numpy as np

# Problem 1

### a)

$dC_t = \frac{\partial C}{\partial r_t}dr_t+\frac{\partial C}{\partial t}dt+\frac{1}{2}\frac{\partial^2 C}{\partial r_t^2}(dr_t)^2$

$= \alpha(r_t,t)\frac{\partial C}{\partial r_t}dt+\beta(r_t,t)\frac{\partial C}{\partial r_t}dW_t+\frac{\partial C}{\partial t}dt+\frac{1}{2}\frac{\partial^2 C}{\partial r_t^2}\beta^2(r_t,t)dt$

$=(\alpha(r_t,t)\frac{\partial C}{\partial r_t}+\frac{1}{2}\frac{\partial^2 C}{\partial r_t^2}\beta^2(r_t,t)+\frac{\partial C}{\partial t})dt+\beta(r_t,t)\frac{\partial C}{\partial r_t}dW_t$

Set drift equal to rC
\begin{align}
&\implies \alpha(r,t)\frac{\partial C}{\partial r}+\frac{1}{2}\frac{\partial^2 C}{\partial r^2}\beta^2(r,t)+\frac{\partial C}{\partial t}=rC(r,t)
\end{align}



### b)
As shown in slides

$q_u = \frac{1}{2}\big[\frac{\kappa(\theta-r_j)\Delta t}{\Delta r}+\frac{\sigma^2\Delta t}{(\Delta r)^2}\big]$

$q_m = 1-\frac{\sigma^2\Delta t}{(\Delta r)^2}$

$q_u = \frac{1}{2}\big[-\frac{\kappa(\theta-r_j)\Delta t}{\Delta r}+\frac{\sigma^2\Delta t}{(\Delta r)^2}\big]$

In [10]:
class Vasicek:
    
    def __init__(self,kappa,theta,sigma): 
        self.kappa=kappa
        self.theta=theta
        self.sigma=sigma

In [11]:
hw3dynamics = Vasicek(kappa=3,theta=0.05,sigma=0.03)

In [12]:
class Bond:
    
    def __init__(self, T):
        self.T=T
    

In [13]:
hw3contract = Bond(T=5)

In [38]:
class FDexplicit:
    
    def __init__(self, rMax, rMin, deltar, deltat, useUpwind):         
        self.rMax=rMax
        self.rMin=rMin
        self.deltar=deltar
        self.deltat=deltat
        self.useUpwind=useUpwind
        
    
    def price_bond_vasicek(self,contract,dynamics):
    # You complete the coding of this function
    #
    # Returns array of all initial short rates,
    # and the corresponding array of zero-coupon
    # T-maturity bond prices
        T = contract.T
        N=round(T/self.deltat)
        if abs(N-T/self.deltat) > 1e-12:
            raise ValueError("Bad delta t")

        r=np.arange(self.rMax,self.rMin-self.deltar/2,-self.deltar)   #I'm making the FIRST indices of the array correspond to HIGH levels of r
        bondprice=np.ones(np.size(r))
        
        a=dynamics.kappa*(dynamics.theta-r)*self.deltat/self.deltar
        b=(dynamics.sigma**2*self.deltat/(self.deltar**2))
        
        if self.useUpwind:
#             print(b,a)
            qu=    0.5*(b+a+abs(a))
            qd=    0.5*(b-a+abs(a))
            qm=    1-qu-qd
        else:
            qu=    0.5*(a+b)
            qm=    1-b
            qd=    0.5*(b-a)

        for t in np.arange(N-1,-1,-1)*self.deltat:
            bondprice=1/(1+r*self.deltat)*(qd*np.roll(bondprice,-1)+qm*bondprice+qu*np.roll(bondprice,1))
            # It is not obvious in this case, 
            # what boundary conditions to use at the top and bottom
            # so let us assume "linearity" boundary conditions
            bondprice[0]=2*bondprice[1]-bondprice[2]
            bondprice[-1]=2*bondprice[-2]-bondprice[-3]

        return (r, bondprice)

In [39]:
hw3FD = FDexplicit(rMax=0.35,rMin=-0.25,deltar=0.01,deltat=0.01,useUpwind=False)

In [40]:
(r, bondprice) = hw3FD.price_bond_vasicek(hw3contract,hw3dynamics)

In [41]:
np.set_printoptions(precision=4,suppress=True)
displayrows=(r<0.15+hw3FD.deltar/2) & (r>0.0-hw3FD.deltar/2)

In [42]:
central_diff_price = np.stack((r, bondprice),1)[displayrows]
print(central_diff_price )

[[ 1.5000e-01 -1.4273e+09]
 [ 1.4000e-01  1.6361e+08]
 [ 1.3000e-01  2.2294e+07]
 [ 1.2000e-01 -1.3724e+06]
 [ 1.1000e-01 -1.3361e+05]
 [ 1.0000e-01  3.2966e+03]
 [ 9.0000e-02  1.3021e+02]
 [ 8.0000e-02  7.7128e-01]
 [ 7.0000e-02  7.7385e-01]
 [ 6.0000e-02  7.7643e-01]
 [ 5.0000e-02  7.7902e-01]
 [ 4.0000e-02  7.8162e-01]
 [ 3.0000e-02  7.8423e-01]
 [ 2.0000e-02  7.8685e-01]
 [ 1.0000e-02  1.4165e+03]
 [-3.3307e-16  5.1498e+04]]


### c)

Case1: $\kappa(\theta-r_j)\ge 0$

$\frac{\partial C}{\partial r}\approx\frac{C^{j+1}_{n+1}-C^{j}_{n+1}}{\Delta r}$

So: 

$q_u = \frac{\kappa(\theta-r_j)\Delta t}{\Delta r}+\frac{\sigma^2\Delta t}{2(\Delta r)^2}$

$q_m = 1-\frac{\sigma^2\Delta t}{(\Delta r)^2}-\frac{\kappa(\theta-r_j)\Delta t}{\Delta r}$

$q_d = \frac{\sigma^2\Delta t}{2(\Delta r)^2}$



Case2: $\kappa(\theta-r_j) < 0$

$\frac{\partial C}{\partial r}\approx\frac{C^{j+1}_{n}-C^{j}_{n-1}}{\Delta r}$

So:

$q_u = \frac{\sigma^2\Delta t}{2(\Delta r)^2}$

$q_m = 1-\frac{\sigma^2\Delta t}{(\Delta r)^2}+\frac{\kappa(\theta-r_j)\Delta t}{\Delta r}$

$q_d = \frac{\sigma^2\Delta t}{2(\Delta r)^2}-\frac{\kappa(\theta-r_j)\Delta t}{\Delta r}$


So:
    
- $q_u = \frac{\sigma^2\Delta t}{2(\Delta r)^2}+(\frac{\kappa(\theta-r_j)\Delta t}{\Delta r})^+$

- $q_m = 1-\frac{\sigma^2\Delta t}{(\Delta r)^2}-\big|\frac{\kappa(\theta-r_j)\Delta t}{\Delta r}\big|=1-q_u-q_d$

- $q_u = \frac{\sigma^2\Delta t}{2(\Delta r)^2}+(-\frac{\kappa(\theta-r_j)\Delta t}{\Delta r})^+$

In [43]:
hw3FD_upwind = FDexplicit(rMax=0.35,rMin=-0.25,deltar=0.01,deltat=0.01,useUpwind=True)
(r, bondprice) = hw3FD_upwind.price_bond_vasicek(hw3contract,hw3dynamics)
np.set_printoptions(precision=4,suppress=True)
displayrows=(r<0.15+hw3FD.deltar/2) & (r>0.0-hw3FD.deltar/2)
upwind_price = np.stack((r, bondprice),1)[displayrows]
print(upwind_price)

[[ 0.15    0.7536]
 [ 0.14    0.7561]
 [ 0.13    0.7586]
 [ 0.12    0.7611]
 [ 0.11    0.7637]
 [ 0.1     0.7662]
 [ 0.09    0.7688]
 [ 0.08    0.7713]
 [ 0.07    0.7739]
 [ 0.06    0.7765]
 [ 0.05    0.7791]
 [ 0.04    0.7817]
 [ 0.03    0.7843]
 [ 0.02    0.7869]
 [ 0.01    0.7895]
 [-0.      0.7922]]


### d)

Taylor's expansion:

$f(x+h)=f(x)+hf'(x)+\frac{1}{2}h^2f''(x)+O(h^3)$

$f(x-h)=f(x)-hf'(x)+\frac{1}{2}h^2f''(x)+O(h^3)$

$\big|\frac{f(x+h)-f(x)}{h}-f'(x)\big|=\frac{1}{2}hf''(x)+\frac{O(h^3)}{h}=O(h)$

$\big|\frac{f(x+h)-f(x-h)}{2h}-f'(x)\big|$ is definitely bounded by a constant times h2, 
$\therefore \big|\frac{f(x+h)-f(x-h)}{2h}-f'(x)\big|==O(h^2)$

### e)

In [51]:
central_diff_price[abs(central_diff_price[:,0]-0.1)<1e-12]

array([[   0.1   , 3296.5929]])

In [57]:
upwind_price[abs(upwind_price[:,0]-0.1)<1e-12]

array([[0.1   , 0.7662]])

The result from central difference is very inaccurate and result from upwind calculation is more accurate. 

### f)

Less ; Greater

### g)

We are using upwind calculation for both case

In [59]:
# r0 = 0.12
pt = upwind_price[abs(upwind_price[:,0]-0.12)<1e-12][0,1]
ytm = np.log(abs(1/pt))/5.0
ytm

0.05458660238621129

In [60]:
#r0 = 0.02
pt =upwind_price[abs(upwind_price[:,0]-0.02)<1e-12][0,1]
ytm = np.log(abs(1/pt))/5.0
ytm

0.04792684524038049

This PDE of r satisfy the OU Process, so the $\theta$ can be treated as the long-term mean of r.
Therefore, the ytm should be converging to its long term mean as of 0.05 from their $r_0$.

# Problem 2

### a)

$\sigma_{imp}=\bar{\sigma_T}=\sqrt{\frac{1}{T}\int_0^T\sigma^2(t)dt}$

This dynamics is capable of generating a non-constant (with respect to T ) term-structure of implied volatility but not generating an implied volatility skew (non-constant with respect to K) 

Because it $\sigma_{imp}$ is a function of t but not K

### b)

In [66]:
# use code from hw1
import numpy as np
from scipy.stats import norm
from scipy.optimize import bisect, brentq
import math
class GBMdynamics: 
    
    def __init__(self, S, r, rGrow, sigma=None):
        self.S = S
        self.r = r
        self.rGrow = rGrow
        self.sigma = sigma
        
    def update_sigma(self, sigma):
        self.sigma = sigma
        return self
class CallOption:
    
    def __init__(self, K, T, price=None):
        self.K = K
        self.T = T
        self.price = price

    def BSprice(self, dynamics):
        # ignores self.price if given, because this function calculates price based on the dynamics 
        
        F = dynamics.S*np.exp(dynamics.rGrow*self.T)
        sd = dynamics.sigma*np.sqrt(self.T)
        d1 = np.log(F/self.K)/sd+sd/2
        d2 = d1-sd
        return np.exp(-dynamics.r*self.T)*(F*norm.cdf(d1)-self.K*norm.cdf(d2))
    
    def IV(self, dynamics):
        # ignores dynamics.sigma, because this function solves for sigma.  
        
        if self.price is None: 
            raise ValueError('Contract price must be given')
    
        df = np.exp(-dynamics.r*self.T)  #discount factor
        F = dynamics.S / df
        lowerbound = np.max([0,(F-self.K)*df])
        C = self.price
        if C<lowerbound:
            return np.nan
        if C==lowerbound:
            return 0
        if C>=F*df:
            return np.nan 
        # C should be np.max([0,(F-self.K)*df]) < C < F*df

        dytry = dynamics
        # We "try" values of sigma until we find sigma that generates price C

        # First find lower and upper bounds
        dytry.sigma = 0.2
        while self.BSprice(dytry)>C:
            dytry.sigma /= 2
        while self.BSprice(dytry)<C:
            dytry.sigma *= 2
        hi = dytry.sigma
        lo = hi/2
                           
        # We have calculated "lo" and "hi" which bound the implied volatility from below and above. 
        # In other words, the implied volatility is somewhere in the interval [lo,hi].
        # Then, to calculate the implied volatility within that interval, 
        # for purposes of this homework, you may either (A) write your own bisection algorithm, 
        # or (B) use scipy.optimize.bisect or (C) use scipy.optimize.brentq
        # You will need to provide lo and hi to those solvers.
        # There are other solvers that do not require you to bound the solution 
        # from below and above (for instance, scipy.optimize.fsolve is a useful solver).  
        # However, if you are able to bound the solution (of a single-variable problem), 
        # then bisection or Brent will be more reliable.
        function = lambda sigma: self.BSprice(dynamics.update_sigma(sigma)) - self.price
        impliedVolatility = brentq(function , a=lo, b=hi, xtol=1e-4)     # you fill this in, using bisect or brentq imported from scipy.optimize,
                                 # or by writing your own bisection algorithm.
        return impliedVolatility

In [67]:
hw1dynamics = GBMdynamics(S=100, sigma=0.4, rGrow=0.05, r=0.05)
contract1 = CallOption(K=100, T=0.1, price = 5.25)
contract2 = CallOption(K=100, T=0.2, price = 7.25)
contract3 = CallOption(K=100, T=0.5, price = 9.5)

In [76]:
print("The Implied Volality at t_0.1 is",contract1.IV(hw1dynamics),'\n',
      "The Implied Volality at t_0.2 is",contract2.IV(hw1dynamics) ,'\n',
      "The Implied Volality at t_0.5 is",contract3.IV(hw1dynamics))


The Implied Volality at t_0.1 is 0.39732050375365563 
 The Implied Volality at t_0.2 is 0.38017110912025387 
 The Implied Volality at t_0.5 is 0.29509733829743456


we have $\sigma_{0.1} = 0.397 ,\sigma_{0.2} = 0.3802,\sigma_{0.5} = 0.295$

A step function suffices:
$\sigma(t)=\begin{cases} 
      \sigma_{0.1} & 0 < t\leq 0.1 \\
      \sqrt{2\sigma_{0.2}^2-\sigma_{0.1}^2} & 0.1 < t \leq 0.2\\
      \sqrt{\frac{1}{3}(5\sigma_{0.5}^2-2\sigma_{0.2}^2)} & 0.2 < t \leq 0.5
\end{cases}$

### c)

$\bar\sigma_{0.4}=\sqrt{\frac{1}{0.4}\int_0^{0.4}\sigma^2(t)dt}$

$=\sqrt{\frac{1}{0.4}*(0.1\sigma_{0.1}^2+0.1(2\sigma_{0.2}^2-\sigma_{0.1}^2)+0.2(\frac{1}{3}(5\sigma_{0.5}^2-2\sigma_{0.2}^2))}$

$=\sqrt{\frac{5}{6}\sigma_{0.5}^2+\frac{1}{6}\sigma^{0.2}_2}$


In [79]:
Imp_V = ((5*0.29509733829743456**2/6)+(1*0.38017110912025387 **2/6))** 0.5 
q2_dynamics = GBMdynamics(S=100, sigma=Imp_V, rGrow=0.05, r=0.05)
contract3 = CallOption(K=100, T=0.4)
print("The price of the European call should be: ",contract3.BSprice(q2_dynamics))

The price of the European call should be:  8.78420254114681
